### Import library

In [93]:
# get data
from binance.spot import Spot 

# handle data
import pyspark.sql as ps
from pyspark.sql.functions import from_unixtime,date_format,from_utc_timestamp
from pyspark.sql.types import DateType

# train data
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../env/app.env")

True

### Setup clients

In [75]:
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)
spark = ps.SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "4096m").\
        getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [104]:
DAY_TO_QUERY = 1800
DURATION_EACH_DAY = 60 * 60 * 24 * 1000

### Fetch data

**Day period**

In [105]:
PERIOD = "1d"

In [110]:
today = datetime.utcnow().replace(hour=7, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000)

start_timestamp = today_timestamp - DAY_TO_QUERY * DURATION_EACH_DAY
end_timestamp = today_timestamp

exchanges = client.exchange_info()
symbols = list(map(lambda item: item.get('symbol'), exchanges.get('symbols')))
columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
for symbol in ["BTCUSDT"]:
    if symbol.endswith('USDT'):
        try:
            for timestamp in range(start_timestamp, end_timestamp, DURATION_EACH_DAY * 100):
                data = client.klines(symbol, PERIOD, limit=100, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY * 100)
                if start_timestamp == timestamp:
                    df = spark.createDataFrame(data, schema=columns)
                else:
                    df = df.union(spark.createDataFrame(data))
                
        except Exception as err:
            print(err)
        
        finally:
            df = df \
                .withColumn("open_time", df["open_time"] / 1000) \
                .withColumn("date", from_unixtime('open_time').cast(DateType()))
            df = df \
                .withColumn("day", date_format(df['date'], 'dd')) \
                .withColumn("month", date_format(df['date'], 'MM')) \
                .withColumn("year", date_format(df['date'], 'yyyy'))

            df.write.option("header",True) \
                 .mode("overwrite") \
                 .partitionBy("day","month","year") \
                 .parquet(f"hdfs://hadoop-namenode:9000/crypto/{symbol}/{PERIOD}.parquet")



### 

In [111]:
symbol = "BTCUSDT"
df_load = spark.read.parquet(f"hdfs://hadoop-namenode:9000/crypto/{symbol}/{PERIOD}.parquet")

In [122]:
df_load.sort(df_load.open_time.desc()).collect()

[Row(open_time=1668643200.0, open='16661.61000000', high='16751.00000000', low='16410.74000000', close='16488.59000000', volume='38629.90424000', close_time=1668729599999, quote_asset_volume='642178387.66170520', number_of_trades=943864, taker_buy_base_asset_volume='19237.88056000', taker_buy_quote_asset_volume='319829127.85544470', ignore='0', date=datetime.date(2022, 11, 17), day=17, month=11, year=2022),
 Row(open_time=1668556800.0, open='16900.57000000', high='17015.92000000', low='16378.61000000', close='16662.76000000', volume='261493.40809000', close_time=1668643199999, quote_asset_volume='4362538338.56532390', number_of_trades=6297404, taker_buy_base_asset_volume='129644.72244000', taker_buy_quote_asset_volume='2162757428.72151800', ignore='0', date=datetime.date(2022, 11, 16), day=16, month=11, year=2022),
 Row(open_time=1668470400.0, open='16617.72000000', high='17134.69000000', low='16527.72000000', close='16900.57000000', volume='282461.84391000', close_time=1668556799999, 

In [ ]:
df_load